In [1]:
input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [2]:
#with open('.day5-input.txt', 'r') as f:
#    input = f.read()

In [3]:
seeds, *rest = input.split('\n\n')
seeds = [ int(x) for x in seeds[7:].split(' ') ]

In [4]:
class Mapping:
    def __init__(self, data):
        self.srcs = []
        self.dsts = []
        self.name, *num_strs = data.strip().split('\n')
        # [dest] [src] [spread]
        nums = [ [int(i) for i in x.split(' ')] for x in num_strs ]
        for dst, src, spread in nums:
            self.dsts.append(range(dst, dst+spread))
            self.srcs.append(range(src, src+spread))
    
    def to(self, seed):
        for src, dst in zip(self.srcs, self.dsts):
            if seed in src:
                return dst[src.index(seed)]
        return seed
                

mappings = [ Mapping(e) for e in rest ]

In [5]:
def lookup(seeds):
    lowest = float('inf')
    for seed in seeds:
        n = seed
        for m in mappings:
            n = m.to(n)
        if n < lowest:
            lowest = n
    return lowest
lookup(seeds)

35

In [6]:
seeds, *_ = input.split('\n\n')
it = iter(seeds[7:].split(' '))
seed_ranges = [ range(int(x), int(x)+int(next(it))) for x in it ]
seed_ranges

[range(79, 93), range(55, 68)]

In [7]:
seed_ranges = sorted(seed_ranges, key=lambda r: r.start)
def shorten(seed_ranges):
    new_ranges = []
    i = 0
    while i < len(seed_ranges):
        c = seed_ranges[i]
        if i+1 >= len(seed_ranges):
            new_ranges.append(c) 
            break
        n = seed_ranges[i+1]
        #print(f"{c} to {n}")
        new_ranges = []
        if c.stop < n.start:
            new_ranges.append(c)
        elif c.stop > n.stop:
            i += 1 # skip next
        elif c.stop < n.stop:
            # they overlap, erge
            new_ranges.append(range(c.start, n.stop))
            i += 1 # skip next
        #print(f"\tnew {new_ranges}")
        i += 1
    if new_ranges != seed_ranges:
        return shorten(new_ranges)
    return new_ranges

shorten(seed_ranges)

[range(55, 68), range(79, 93)]

In [8]:
#import itertools
#lookup(itertools.chain(*seed_ranges)) # oh god slow

In [9]:
import asyncio
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp

In [10]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

async def main():
    loop = asyncio.get_running_loop()
    
    futures = []

    with ProcessPoolExecutor(5, mp_context=mp.get_context('fork')) as pool:
        for e in seed_ranges:
            for part in split(e, 3): # 3 parts per, so len(seed_ranges)*3
                #print(f"Adding {part} to task... for {alookup}")
                r = loop.run_in_executor(pool, lookup, part)
                futures.append(r)
    
    lowest = await asyncio.gather(*futures)
    print(min(lowest))

In [11]:
await main() # still takes like an hour

46
